In [3]:
import pandas as pd

In [ ]:
import bamboolib as bam

In [20]:
df = pd.read_csv(r'C:\Users\Admin\Documents\Datium\DatiumSample.rpt', sep ='\t')

# Check missing value and drop features with more than 50% missing value

In [21]:
missing = df.isna().sum().div(df.shape[0]).mul(100).to_frame().sort_values(by=0, ascending = False)

In [22]:
cols_missing = missing[missing[0]>50]
print("There are {} features with NaNs above 50%.".format(len(cols_missing)))

There are 36 features with NaNs above 50%.


In [23]:
# get the list of features to drop due to significant number of missing value
cols_to_drop = list(cols_missing.index)

In [9]:
df = df.drop(columns=cols_to_drop)

# Calculate the median value of feature for each car model

In [10]:
modelfeature = df.groupby(['Model']).agg(FuelUrbanModel=('FuelUrban', 'median'), 
                                         FuelExtraurbanModel=('FuelExtraurban', 'median'),
                                         PayLoadModel=('PayLoad', 'median'), 
                                         TareMassModel=('TareMass', 'median'),
                                         KerbWeightModel=('KerbWeight', 'median'), 
                                         RonRatingModel=('RonRating', 'median'),
                                         GrossVehicleMassModel=('GrossVehicleMass', 'median'), 
                                         GreenhouseRatingModel=('GreenhouseRating', 'median'),
                                         AirpollutionRatingModel=('AirpollutionRating', 'median'), 
                                         OverallGreenStarRatingModel=('OverallGreenStarRating', 'median'),
                                         FuelCombinedModel=('FuelCombined', 'median'), 
                                         CO2CombinedModel=('CO2Combined', 'median'),
                                         AncapRatingModel=('AncapRating', 'median')).reset_index()

# Join with original df, replace missing value with median value of the car model

In [ ]:
df = pd.merge(df, modelfeature.drop(columns=[]), how='left', on=['Model'])
df['FuelUrbanFinal'] = df['FuelUrban'].combine_first(df['FuelUrbanModel'])
df['FuelExtraurbanFinal'] = df['FuelExtraurban'].combine_first(df['FuelExtraurbanModel'])
df['PayLoadFinal'] = df['PayLoad'].combine_first(df['PayLoadModel'])
df['TareMassFinal'] = df['TareMass'].combine_first(df['TareMassModel'])
df['KerbWeightFinal'] = df['KerbWeight'].combine_first(df['KerbWeightModel'])
df['GrossVehicleMassFinal'] = df['GrossVehicleMass'].combine_first(df['GrossVehicleMassModel'])
df['RonRatingFinal'] = df['RonRating'].combine_first(df['RonRatingModel'])
df['GreenhouseRatingFinal'] = df['GreenhouseRating'].combine_first(df['GreenhouseRatingModel'])
df['AirpollutionRatingFinal'] = df['AirpollutionRating'].combine_first(df['AirpollutionRatingModel'])
df['OverallGreenStarRatingFinal'] = df['OverallGreenStarRating'].combine_first(df['OverallGreenStarRatingModel'])
df['FuelCombinedFinal'] = df['FuelCombined'].combine_first(df['FuelCombinedModel'])
df['CO2CombinedFinal'] = df['CO2Combined'].combine_first(df['CO2CombinedModel'])
df['AncapRatingFinal'] = df['AncapRating'].combine_first(df['AncapRatingModel'])

In [ ]:
col_drop = ['VIN','ModelCode','EngineNum','Sold_Date','Compliance_Date', 'AvgWholesale', 'AvgRetail', 'GoodWholesale', 
            'GoodRetail', 'TradeMin', 'TradeMax', 'PrivateMax','FuelUrban', 'FuelExtraurban', 'PayLoad', 'TareMass','KerbWeight',
           'FuelUrbanModel', 'FuelExtraurbanModel', 'PayLoadModel', 'TareMassModel','KerbWeightModel','GrossVehicleMassModel', 
           'GrossVehicleMass', 'Make','Model', 'Description', 'RonRating','RonRatingModel','GreenhouseRating','GreenhouseRatingModel',
           'AirpollutionRating', 'AirpollutionRatingModel', 'OverallGreenStarRating', 'OverallGreenStarRatingModel', 'FuelCombined', 
           'FuelCombinedModel','CO2Combined','CO2CombinedModel','AncapRating','AncapRatingModel']
df = df.drop(columns=col_drop)

In [ ]:
df['Series'] = df['Series'].fillna('No Series')
df['BadgeDescription'] = df['BadgeDescription'].fillna('No BadgeDescription')
df['VFactsPrice'] = df['VFactsPrice'].fillna('No VFactsPrice')


In [ ]:
df = df.loc[df['Sold_Amount'].isna() == False]

In [ ]:
df

In [ ]:
df.to_csv('Clean_df.csv', index = False)

In [4]:
df = pd.read_csv(r'C:\Users\Admin\Documents\Datium\Clean_df.csv')

# Model training

In [12]:
from pycaret.regression import * 

In [24]:
reg1 = setup(data = df, target = 'Sold_Amount', categorical_imputation = 'constant', 
            numeric_imputation = 'median', normalize = True, combine_rare_levels = True, 
            remove_multicollinearity = True, # pca = True, pca_components = 0.95,feature_selection_threshold = 0.7, 
            log_experiment = True, experiment_name = 'datium',  train_size = 0.75,
            feature_selection = True, ignore_low_variance = True, remove_outliers = True,
            high_cardinality_features = ['MakeCode','FamilyCode','Series','BadgeDescription','EngineDescription',
                                         'FrontTyreSize', 'RearTyreSize', 'FrontRimDesc','RearRimDesc', 'Colour']) 

,Description,Value
0,session_id,5526
1,Target,Sold_Amount
2,Original Data,"(62188, 79)"
3,Missing Values,True
4,Numeric Features,31
5,Categorical Features,47
6,Ordinal Features,False
7,High Cardinality Features,True
8,High Cardinality Method,frequency
9,Transformed Train Set,"(44309, 229)"


In [ ]:
best = compare_models()

In [ ]:
evaluate_model(cat)

In [14]:
cat = create_model('catboost')

,MAE,MSE,RMSE,R2,RMSLE,MAPE
0,1540.9150,5430938.4877,2330.4374,0.9385,0.9325,0.1246
1,1541.1770,5737085.8263,2395.2215,0.9278,0.8969,0.1234
2,1552.6257,5431857.1387,2330.6345,0.9363,0.9532,0.1365
3,1521.7017,5413922.4554,2326.7837,0.9385,0.9527,0.1310
4,1543.8458,5499440.4743,2345.0886,0.9371,0.9123,0.1274
5,1536.0868,5179425.1118,2275.8350,0.9382,0.8738,0.1409
6,1523.9578,5648766.0152,2376.7133,0.9337,0.9275,0.1199
7,1585.5985,6012544.1848,2452.0490,0.9327,0.9413,0.1213
8,1529.7890,5802153.2476,2408.7659,0.9394,0.9119,0.1228
9,1524.2602,5785914.0881,2405.3927,0.9375,0.9923,0.1182


In [ ]:
pred_holdout = predict_model(cat)

In [ ]:
pred_holdout.tail(25)

In [ ]:
save_model(cat, r'C:\Users\Admin\Documents\Datium\Experiment\Run 5\Cat_Boost')

In [ ]:
file_loc = (r'C:\Users\Admin\Documents\Datium\Experiment\Run 5\Run5vars.pkl')
save_config(file_loc)

In [13]:
load_config(r'C:\Users\Admin\Documents\Datium\Experiment\Run 5\Run5vars.pkl')

In [17]:
cat_saved = load_model(r'C:\Users\Admin\Documents\Datium\Experiment\Run 5\Cat_Boost')

Transformation Pipeline and Model Successfully Loaded


In [18]:
cat_saved

Pipeline(memory=None,
         steps=[('dtypes',
                 DataTypes_Auto_infer(categorical_features=[],
                                      display_types=True, features_todrop=[],
                                      id_columns=[], ml_usecase='regression',
                                      numerical_features=[],
                                      target='Sold_Amount', time_features=[])),
                ('imputer',
                 Simple_Imputer(categorical_strategy='not_available',
                                fill_value_categorical=None,
                                fill_value_numerical=None,
                                numeric_stra...
                                                    target='Sold_Amount',
                                                    top_features_to_pick=0.30000000000000004)),
                ('fix_multi',
                 Fix_multicollinearity(correlation_with_target_preference=None,
                                       correl

# Demo

In [27]:
demo = pd.read_csv(r'C:\Users\Admin\Documents\Datium\Clean_df.csv')
median_value = pd.read_csv(r'C:\Users\Admin\Documents\modelfeature_median.csv')
df = pd.read_csv(r'C:\Users\Admin\Documents\Datium\DatiumSample.rpt', sep ='\t')

In [26]:
drop_col = ['AltEngTorqueFrom', 'QuickChargeVoltage', 'QuickChargeMins', 'NormalChargeVoltage', 
            'NormalChargeMins', 'AltEngAmpHours', 'TopSpeedElectricEng', 'KMRangeElectricEng',
            'ElectricEngineLocation', 'AltEngPowerFrom', 'AltEngTorqueTo', 'FreeScheduledService',
            'AltEngPowerTo', 'PowerRPMFrom', 'Roofline', 'AltEngTorque', 'AltEngDrive', 'AltEngPower',
            'AltEngChargingMethod', 'AltEngCurrentType', 'AltEngBatteryType', 'AltEngEngineType',
            'AltEngVolts', 'WheelBaseConfig', 'BadgeSecondaryDescription', 'ExtraIdentification',
            'MaxEthanolBlend', 'TorqueRPMFrom', 'BodyConfigDescription', 'EmissionStandard',
            'Acceleration', 'CO2ExtraUrban', 'CO2Urban', 'SeriesModelYear', 'WarrantyCustAssist',
            'GrossCombinationMAss']

In [28]:
df = df.drop(columns=drop_col)

In [29]:
df = pd.merge(df, median_value.drop(columns=[]), how='left', on=['Model'])
df['FuelUrbanFinal'] = df['FuelUrban'].combine_first(df['FuelUrbanModel'])
df['FuelExtraurbanFinal'] = df['FuelExtraurban'].combine_first(df['FuelExtraurbanModel'])
df['PayLoadFinal'] = df['PayLoad'].combine_first(df['PayLoadModel'])
df['TareMassFinal'] = df['TareMass'].combine_first(df['TareMassModel'])
df['KerbWeightFinal'] = df['KerbWeight'].combine_first(df['KerbWeightModel'])
df['GrossVehicleMassFinal'] = df['GrossVehicleMass'].combine_first(df['GrossVehicleMassModel'])
df['RonRatingFinal'] = df['RonRating'].combine_first(df['RonRatingModel'])
df['GreenhouseRatingFinal'] = df['GreenhouseRating'].combine_first(df['GreenhouseRatingModel'])
df['AirpollutionRatingFinal'] = df['AirpollutionRating'].combine_first(df['AirpollutionRatingModel'])
df['OverallGreenStarRatingFinal'] = df['OverallGreenStarRating'].combine_first(df['OverallGreenStarRatingModel'])
df['FuelCombinedFinal'] = df['FuelCombined'].combine_first(df['FuelCombinedModel'])
df['CO2CombinedFinal'] = df['CO2Combined'].combine_first(df['CO2CombinedModel'])
df['AncapRatingFinal'] = df['AncapRating'].combine_first(df['AncapRatingModel'])

In [30]:
col_drop = ['VIN','ModelCode','EngineNum','Sold_Date','Compliance_Date', 'AvgWholesale', 'AvgRetail', 'GoodWholesale', 
            'GoodRetail', 'TradeMin', 'TradeMax', 'PrivateMax','FuelUrban', 'FuelExtraurban', 'PayLoad', 'TareMass','KerbWeight',
           'FuelUrbanModel', 'FuelExtraurbanModel', 'PayLoadModel', 'TareMassModel','KerbWeightModel','GrossVehicleMassModel', 
           'GrossVehicleMass', 'Make','Model', 'Description', 'RonRating','RonRatingModel','GreenhouseRating','GreenhouseRatingModel',
           'AirpollutionRating', 'AirpollutionRatingModel', 'OverallGreenStarRating', 'OverallGreenStarRatingModel', 'FuelCombined', 
           'FuelCombinedModel','CO2Combined','CO2CombinedModel','AncapRating','AncapRatingModel']
df = df.drop(columns=col_drop)

In [31]:
df['Series'] = df['Series'].fillna('No Series')
df['BadgeDescription'] = df['BadgeDescription'].fillna('No BadgeDescription')
df['VFactsPrice'] = df['VFactsPrice'].fillna('No VFactsPrice')

In [32]:
demo = df.iloc[2350:2370]

In [34]:
cat_saved = load_model(r'C:\Users\Admin\Documents\Datium\Experiment\Run 5\Cat_Boost')

Transformation Pipeline and Model Successfully Loaded


In [37]:
predictions = predict_model(cat_saved, data = demo)

In [38]:
predictions[['NewPrice','KM','Age_Comp_Months','Height','Sold_Amount','Label']]

,NewPrice,KM,Age_Comp_Months,Height,Sold_Amount,Label
2350,29800.0,66036.0,41.0,1435.0,12800.0,11994.355948
2351,54050.0,151042.0,42.0,2115.0,13000.0,13363.250515
2352,28104.0,34772.0,31.0,1445.0,18400.0,17525.144528
2353,29990.0,46065.0,25.0,1480.0,14000.0,15491.940384
2354,87588.0,58133.0,25.0,1735.0,49500.0,49862.359579
2355,45990.0,62754.0,33.0,1490.0,24500.0,22972.347131
2356,38490.0,69861.0,37.0,1476.0,17000.0,14307.986560
2357,27990.0,93574.0,30.0,1476.0,13250.0,14572.313774
2358,34490.0,42844.0,15.0,1476.0,20300.0,19787.972266
2359,33990.0,196862.0,58.0,1775.0,9200.0,9649.676818
